In [2]:
from bs4 import BeautifulSoup
import requests as r

url= "https://myanimelist.net/topanime.php?limit=0"
page = r.get(url)
page 

<Response [200]>

In [8]:
soup = BeautifulSoup(page.text, 'html.parser')

In [13]:
ranking_table = soup.find("table", "top-ranking-table")

In [14]:
ranking_entries = ranking_table.find_all("tr", "ranking-list")

In [78]:
for entry in ranking_entries[:10]:
    entry_h3 = entry.find("h3", "anime_ranking_h3")
    entry_link = entry_h3.find("a")["href"]
    entry_name = entry_h3.text
    entry_soup = BeautifulSoup(r.get(entry_link).text, 'html.parser')
    if entry_soup.find("span", string="Type:"):
        entry_type = entry_soup.find("span", string="Type:").next_sibling.next_sibling.text
    if entry_soup.find("span", string="Aired:"):
        entry_aired = entry_soup.find("span", string="Aired:").next_sibling.text.split("to")
        entry_begin = entry_aired[0].strip()
        entry_end = entry_aired[1].strip()
    if entry_soup.find("span", string="Episodes:"):
        entry_episodes = entry_soup.find("span", string="Episodes:").next_sibling.text.strip()
    if entry_soup.find("span", string="Studios:"):
        entry_studio = entry_soup.find("span", string="Studios:").next_sibling.next_sibling.text
    if entry_soup.find("span", string="Demographic:"):
        entry_demographic = entry_soup.find("span", string="Demographic:").next_sibling.next_sibling.text
    if entry_soup.find("span", string="Premiered:"):
        entry_season = entry_soup.find("span", string="Premiered:").next_sibling.next_sibling.text
    if entry_soup.find("span", string="Genres:"):
        entry_genres = [genre_item.text for genre_item in entry_soup.find("span", string="Genres:").find_next_siblings("a")]
    elif entry_soup.find("span", string="Genre:"):
        entry_genres = [genre_item.text for genre_item in entry_soup.find("span", string="Genre:").find_next_siblings("a")]
    if entry_soup.find("span", string="Themes:"):
        entry_themes = [theme_item.text for theme_item in entry_soup.find("span", string="Themes:").find_next_siblings("a")]
    elif entry_soup.find("span", string="Theme:"):
        entry_themes = [theme_item.text for theme_item in entry_soup.find("span", string="Theme:").find_next_siblings("a")]
    if entry_soup.find("span", {"itemprop": "ratingValue"}):
        entry_score = entry_soup.find("span", {"itemprop": "ratingValue"}).text
    if entry_soup.find("span", {"itemprop": "ratingCount"}):
        entry_rating_count = entry_soup.find("span", {"itemprop": "ratingCount"}).text
    if entry_soup.find("span", string="Popularity:"):
        entry_popularity = entry_soup.find("span", string="Popularity:").next_sibling.text.strip().replace("#", "")
    if entry_soup.find("span", string="Favorites:"):
        entry_favorites = entry_soup.find("span", string="Favorites:").next_sibling.text.strip() 
    if entry_soup.find("span", string="Members:"):
        entry_members = entry_soup.find("span", string="Members:").next_sibling.text.strip() 

9.12
9.09
9.07


KeyboardInterrupt: 

In [87]:
from bs4 import BeautifulSoup
import requests as r

class MALExtractor:
    
    def getTopAnimesUpToPage(self, pageNum):
        """
        Gets a dictionary of anime information from the MyAnimeList top animes leaderboard up to a given rank.
        Parameter:
            pageNum: the number of pages from the top animes list to query
        Return:
            Returns a dictionary containing information on the top n animes on the MyAnimeList top animes leaderboard
            Included fields: "name", "link", "
        """

        url = "https://myanimelist.net/topanime.php?limit="
        animeData = []

        for i in range(pageNum):
            rankingEntries = self.getPageAnimeEntries(url + str(i * 50))
            for entry in rankingEntries:
                animeURL = self.getAnimeURL(entry)
                animeData.append(self.getDataFromAnimeURL(animeURL))
        return animeData
    

    def getAnimeURL(self, rankingEntry):
        entryH3 = rankingEntry.find("h3", "anime_ranking_h3")
        return entryH3.find("a")["href"]

    def getPageAnimeEntries(self, pageURL):
        pageSoup = BeautifulSoup(r.get(pageURL).text, "html.parser")
        rankingTable = pageSoup.find("table", "top-ranking-table")
        return rankingTable.find_all("tr", "ranking-list")

    def getDataFromAnimeURL(self, animeURL):
        print(animeURL)
        entrySoup = BeautifulSoup(r.get(animeURL).text, "html.parser")
        return {
            "Name": self.getAnimeName(entrySoup),
            "Type": self.getAnimeType(entrySoup),
            "Score": self.getAnimeScore(entrySoup),
            "Begin": self.getAnimeBegin(entrySoup),
            "End": self.getAnimeEnd(entrySoup),
            "Episodes": self.getAnimeEpisodes(entrySoup),
            "Studios": self.getAnimeStudios(entrySoup),
            "Demographic": self.getAnimeDemographic(entrySoup),
            "Season": self.getAnimeSeason(entrySoup),
            "Genres": self.getAnimeGenres(entrySoup),
            "Themes": self.getAnimeThemes(entrySoup),
            "RatingCount": self.getAnimeRatingCount(entrySoup),
            "Members": self.getAnimeMembers(entrySoup),
            "Favorites": self.getAnimeFavorites(entrySoup),
            "Popularity": self.getAnimePopularity(entrySoup),
            "Link": animeURL,
        }
    
    def getAnimeName(self, entrySoup):
        if entrySoup.find("h1", "title-name"):
           return entrySoup.find("h1", "title-name").text
    
    def getAnimeType(self, entrySoup):
        if entrySoup.find("span", string="Type:"):
           return entrySoup.find("span", string="Type:").next_sibling.next_sibling.text
        
    def getAnimeBegin(self, entrySoup):
        if entrySoup.find("span", string="Aired:"):
            entryAired = entrySoup.find("span", string="Aired:").next_sibling.text.split("to")
            return entryAired[0].strip()
    
    def getAnimeEnd(self, entrySoup):
        if entrySoup.find("span", string="Aired:"):
            entryAired = entrySoup.find("span", string="Aired:").next_sibling.text.split("to")
            if len(entryAired) > 1:
                return entryAired[1].strip()
            else:
                return "n/a"
        
    def getAnimeEpisodes(self, entrySoup):
        if entrySoup.find("span", string="Episodes:"):
            return entrySoup.find("span", string="Episodes:").next_sibling.text.strip()
        
    def getAnimeStudios(self, entrySoup):
        if entrySoup.find("span", string="Studios:"):
            return entrySoup.find("span", string="Studios:").next_sibling.next_sibling.text
        
    def getAnimeDemographic(self, entrySoup):
        if entrySoup.find("span", string="Demographic:"):
            return entrySoup.find("span", string="Demographic:").next_sibling.next_sibling.text
        
    def getAnimeSeason(self, entrySoup):
        if entrySoup.find("span", string="Premiered:"):
            return entrySoup.find("span", string="Premiered:").next_sibling.next_sibling.text

    def getAnimeGenres(self, entrySoup): 
        if entrySoup.find("span", string="Genres:"):
            return [genre_item.text for genre_item in entrySoup.find("span", string="Genres:").find_next_siblings("a")]
        elif entrySoup.find("span", string="Genre:"):
            return [genre_item.text for genre_item in entrySoup.find("span", string="Genre:").find_next_siblings("a")]
        
    def getAnimeThemes(self, entrySoup):
        if entrySoup.find("span", string="Themes:"):
            return [theme_item.text for theme_item in entrySoup.find("span", string="Themes:").find_next_siblings("a")]
        elif entrySoup.find("span", string="Theme:"):
            return [theme_item.text for theme_item in entrySoup.find("span", string="Theme:").find_next_siblings("a")]
        
    def getAnimeScore(self, entrySoup):
        if entrySoup.find("span", {"itemprop": "ratingValue"}):
            return entrySoup.find("span", {"itemprop": "ratingValue"}).text
        
    def getAnimeRatingCount(self, entrySoup):
        if entrySoup.find("span", {"itemprop": "ratingCount"}):
            return entrySoup.find("span", {"itemprop": "ratingCount"}).text
        
    def getAnimePopularity(self, entrySoup):
        if entrySoup.find("span", string="Popularity:"):
            return entrySoup.find("span", string="Popularity:").next_sibling.text.strip().replace("#", "")
        
    def getAnimeFavorites(self, entrySoup):
        if entrySoup.find("span", string="Favorites:"):
            return entrySoup.find("span", string="Favorites:").next_sibling.text.strip() 
        
    def getAnimeMembers(self, entrySoup):
        if entrySoup.find("span", string="Members:"):
            return entrySoup.find("span", string="Members:").next_sibling.text.strip() 

In [88]:
male = MALExtractor()
male.getTopAnimesUpToPage(1)

https://myanimelist.net/anime/52991/Sousou_no_Frieren
https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood
https://myanimelist.net/anime/9253/Steins_Gate
https://myanimelist.net/anime/28977/Gintama°
https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2
https://myanimelist.net/anime/41467/Bleach__Sennen_Kessen-hen
https://myanimelist.net/anime/39486/Gintama__The_Final
https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011
https://myanimelist.net/anime/9969/Gintama
https://myanimelist.net/anime/15417/Gintama__Enchousen
https://myanimelist.net/anime/43608/Kaguya-sama_wa_Kokurasetai__Ultra_Romantic
https://myanimelist.net/anime/820/Ginga_Eiyuu_Densetsu
https://myanimelist.net/anime/42938/Fruits_Basket__The_Final
https://myanimelist.net/anime/34096/Gintama
https://myanimelist.net/anime/51535/Shingeki_no_Kyojin__The_Final_Season_-_Kanketsu-hen
https://myanimelist.net/anime/918/Gintama
https://myanimelist.net/anime/35180/3-gatsu_no_Lion_2nd_Season
https://mya

[{'Name': 'Sousou no Frieren',
  'Type': 'TV',
  'Score': '9.12',
  'Begin': 'Sep 29, 2023',
  'End': '?',
  'Episodes': '28',
  'Studios': 'Madhouse',
  'Demographic': 'Shounen',
  'Season': 'Fall 2023',
  'Genres': ['Adventure', 'Drama', 'Fantasy'],
  'Themes': None,
  'RatingCount': '89712',
  'Members': '343,524',
  'Favorites': '8,121',
  'Popularity': '664',
  'Link': 'https://myanimelist.net/anime/52991/Sousou_no_Frieren'},
 {'Name': 'Fullmetal Alchemist: Brotherhood',
  'Type': 'TV',
  'Score': '9.09',
  'Begin': 'Apr 5, 2009',
  'End': 'Jul 4, 2010',
  'Episodes': '64',
  'Studios': 'Bones',
  'Demographic': 'Shounen',
  'Season': 'Spring 2009',
  'Genres': ['Action', 'Adventure', 'Drama', 'Fantasy'],
  'Themes': ['Military'],
  'RatingCount': '2071028',
  'Members': '3,262,478',
  'Favorites': '221,892',
  'Popularity': '3',
  'Link': 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood'},
 {'Name': 'Steins;Gate',
  'Type': 'TV',
  'Score': '9.07',
  'Begin': 